<a href="https://colab.research.google.com/github/rajeshsahu09/CS69002_9A_18CS60R19/blob/master/DL_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Review Sentiment Analysis

### Import Header Files

In [0]:
import torch
import pandas as pd
import numpy as np

### Load the Dataset and Visualise

In [0]:
from google.colab import files
uploaded = files.upload()

In [6]:
import io
df = pd.read_csv(io.StringIO(uploaded['Train_20K.csv'].decode('utf-8')), sep='\t')
df.head()

,text,label
0,John Waters has given us a genuinely enjoyable...,1
1,This first two seasons of this comedy series w...,1
2,"What an unfortunate mess is ""Shiner."" I wanted...",0
3,I'm not entirely sure Rob Schmidt qualifies as...,1
4,i wasn't sure whether to laugh or cry. Porrett...,0


In [18]:
df[df['label']==0]

,text,label
2,"What an unfortunate mess is ""Shiner."" I wanted...",0
4,i wasn't sure whether to laugh or cry. Porrett...,0
6,A friend once asked me to read a screenplay of...,0
7,........and an extremely bad one at that!!! Ho...,0
9,After having seen the movie the first question...,0
10,There are so many puns to play on the title of...,0
13,What a shame that a really competent director ...,0
16,86 wasted minutes of my life. I fell asleep th...,0
21,This show is unbelievable in that . . . what i...,0
23,"The hysterical thing about this movie is that,...",0


In [19]:
print('Number of Negative movie reviews', len(df[df['label']==0]))
print('Number of Positive movie reviews', len(df[df['label']==1]))

Number of Negative movie reviews 8994
Number of Positive movie reviews 9005


### Data pre-processing

In [77]:
raw_text_reviews = df['text'].astype(str).tolist()
len(raw_text_reviews), raw_text_reviews[-2]

(17999,
 'Anyone notice that Tommy only has 3 facial expressions.<br /><br />1. The angry eyes look he gives every enemy. 2. The holding of the hands to face, mouth agape and frightened eyes. 3. The smiling Tommy Turnbull.<br /><br />I have to say that i pretty much hate this show, i don\'t watch it but it\'s like Code Lyoko, we\'ve all watched at least one, i must say that this show is borderline racist, uninteresting and pointless.<br /><br />every episode ends with robotboy winning, except for one exception when robotboy basically let this overly geeky freakazoid fly away on a jetpack.<br /><br />The jokes are pretty crude too, i think it\'s mostly people saying the word "Suck" or farting, i think the bullies of the show are quite shocking too.<br /><br />Isn\'t there one that hides a bowling ball under his hate, and the other uses a chain, for god sake, what kind of school is he going to. Not to mention his older brother, who is borderline psychopathic and has no other character qu

In [78]:
text_labels = df['label'].astype(int)
len(text_labels), text_labels[len(text_labels)-2]

(17999, 0)

In [0]:
import re
import string
def preprocess_document(doc):
  # negative sense should not be eleminated + some short representation
  CONTRACTIONS = {"mayn't":"may not", "can't":"can not", "won't":"will not", "isn't":"is not", "amn't":"am not",\
                  "aren't":"are not", "wasn't":"was not", "weren't":"were not", "couldn't":"could not", \
                  "wouldn't":"would not", "don't":"do not", "doesn't":"does not",\
                  "i'll":"i will", "you'll":"you will", "they'll":"they have",\
                  "may've":"may have", "can've":"can have", "will've":"will have", "you've":"you have", \
                  "could've":"could have", "would've":"would have", "they":"they have",\
                  "i've":"i have", "you've":"you have", "we've":"we have", "there's":"there is", "i'am":"i am", "it's":"it is"}
  punctuation = string.punctuation
  punc_replace = ''.join([' ' for s in punctuation]); # required for replacing punctuation with null ('')
  doc_clean = doc.replace('-', ' '); # replace - with null str
  doc_clean = doc_clean.replace('<br />', ''); # replace <br /> with ''
  doc_clean = doc_clean.replace("’", "'"); # replace <br /> with null str
  doc_clean = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in doc_clean.split(' ')] # replacing some common short forms
  doc_clean = " ".join(doc_clean) # list to sentence of strings
  doc_clean = re.sub(r'\W +', '', doc_clean) # except [a-zA-Z0-9_]
  doc_clean = re.sub(r'\d+', '', doc_clean) # remove numbers [0-9]
  trans_table = str.maketrans(punctuation, punc_replace); # replace punctuations with ' '
  doc_clean = ' '.join([word.translate(trans_table) for word in doc_clean.split(' ')]);
  doc_clean = doc_clean.split(' ');
  doc_clean = [word for word in doc_clean if len(word) > 0];

  return doc_clean;

In [84]:
train_text_reviews = [preprocess_document(review.lower()) for review in raw_text_reviews]
print (train_text_reviews[len(train_text_reviews)-2])
print (text_labels[len(text_labels)-2])

['anyone', 'notice', 'that', 'tommy', 'only', 'has', 'facial', 'expressions', 'the', 'angry', 'eyes', 'look', 'he', 'gives', 'every', 'enemythe', 'holding', 'of', 'the', 'hands', 'to', 'facemouth', 'agape', 'and', 'frightened', 'eyesthe', 'smiling', 'tommy', 'turnbull', 'i', 'have', 'to', 'say', 'that', 'i', 'pretty', 'much', 'hate', 'this', 'showi', 'do', 'not', 'watch', 'it', 'but', 'it', 'is', 'like', 'code', 'lyokowe', 'have', 'all', 'watched', 'at', 'least', 'onei', 'must', 'say', 'that', 'this', 'show', 'is', 'borderline', 'racistuninteresting', 'and', 'pointless', 'every', 'episode', 'ends', 'with', 'robotboy', 'winningexcept', 'for', 'one', 'exception', 'when', 'robotboy', 'basically', 'let', 'this', 'overly', 'geeky', 'freakazoid', 'fly', 'away', 'on', 'a', 'jetpack', 'the', 'jokes', 'are', 'pretty', 'crude', 'tooi', 'think', 'it', 'is', 'mostly', 'people', 'saying', 'the', 'word', 'suckor', 'fartingi', 'think', 'the', 'bullies', 'of', 'the', 'show', 'are', 'quite', 'shocking'

###Creating Bag Of Word (BOW) representation of sentences.

In [90]:
# Fetching the tokens (all words in a doc) and types(unique words in a doc) in the document
types = ()
tokens = []
for row in train_text_reviews:
    tokens += [words for words in row]
types = set(tokens)

vocab_size, token_size = len(types), len(tokens)
vocab_size, token_size

(271920, 3887769)

In [91]:
char_to_ix = {x:i for i, x in enumerate(types)}
ix_to_char = [x for i, x in enumerate(types)]

ix_to_char[char_to_ix['kick']], char_to_ix['kick'], len(ix_to_char)

('kick', 26112, 271920)

In [101]:
for i, (key, value) in enumerate(char_to_ix.items()):
  if(i > 10):
    break
  print (key,':', value)

houseled : 0
earlierdo : 1
trappedkinda : 2
othergiving : 3
mentalitythis : 4
filmjoe : 5
heterosexuals : 6
quatermaincompared : 7
farrellythe : 8
sergius : 9
vmikelsfor : 10
